# Practical Implementation Of PCA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

### Import  the libraries


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Loading The Dataset

To import the dataset I have used the pandas library

In [ ]:
df = pd.read_csv('../input/glass.csv')
df

In [ ]:
columns_names=df.columns.tolist()
print("Columns names:")
print(columns_names)

In [ ]:
df.shape


In [ ]:
df.head()

df.corr() compute pairwise correlation of columns.Correlation shows how the two variables are related to each other.Positive values shows as one variable increases other variable increases as well. Negative values shows as one variable increases other variable decreases.Bigger the values,more strongly two varibles are correlated and viceversa.

In [ ]:
df.corr()

Visualising correlation using Seaborn library

In [ ]:
correlation = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(correlation, vmax=1, square=True,annot=True,cmap='cubehelix')

plt.title('Correlation between different fearures')


##### DataFrame.iloc
Purely integer-location based indexing for selection by position.

.iloc[ ] is primarily integer position based (from 0 to length-1 of the axis)

In [ ]:
X = df.iloc[:,0:11].values
y = df.iloc[:,0].values
X



In [ ]:
np.shape(X)

In [ ]:
y


##  Data Standardisation
Standardization refers to shifting the distribution of each attribute to have a mean of zero and a standard deviation of one (unit variance). It is useful to standardize attributes for a model. Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data

In [ ]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

## Computing Eigenvectors and Eigenvalues:


### Covariance Matrix:

In [ ]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

In [ ]:
print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))


In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(cov_mat, vmax=1, square=True,annot=True,cmap='cubehelix')

plt.title('Correlation between different features')


### Eigen decomposition of the covariance matrix:

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

Sorting The Covariance matrix

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

In [ ]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]

In order to decide which eigenvector(s) can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.

Explained Variance is a useful measure which can be calculated from the eigenvalues. The explained variance tells us how much information (variance) can be attributed to each of the principal components.



In [ ]:
with plt.style.context('dark_background'):
    plt.figure(figsize=(6, 4))

    plt.bar(range(10), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()



The plot above clearly shows that maximum variance (somewhere around 30%) can be explained by the first principal component alone. Comparatively 7th and 8th components share less amount of information as compared to the rest of the Principal components.

### Projection Matrix

In [ ]:
significance = [np.abs(i)/np.sum(eig_vals) for i in eig_vals]

#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(significance))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Explained Variance')
plt.show()

The construction of the projection matrix that will be used to transform the Glass data onto the new feature subspace. From the above bargraph and curve we can observe that the first 5 principal components carry 90% of the data. Hence we can drop other components. Here, we are reducing the 10-dimensional feature space to a 5-dimensional feature subspace, by choosing the “top 5” eigenvectors with the highest eigenvalues to construct our d×k-dimensional eigenvector matrix W

In [ ]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(10,1), 
                      eig_pairs[1][1].reshape(10,1),
                      eig_pairs[2][1].reshape(10,1),
                      eig_pairs[3][1].reshape(10,1),
                      eig_pairs[4][1].reshape(10,1)
                    ))
print('Matrix W:\n', matrix_w)



In [ ]:
np.shape(matrix_w)

In [ ]:
Y = X_std.dot(matrix_w)
Y

Now this projected data can be further used to get accurate results as we have reduced the dimesnions by selecting only the principal components that have max information about data